In [ ]:
import numpy as np
import open3d as o3d
import matplotlib.pyplot as plt 

In [ ]:
DATANAME = "sedeCRO.ply"
data_folder = "cloud_points/"
pcd = o3d.io.read_point_cloud(data_folder + DATANAME)

# Pre Processamento
pcd_center = pcd.get_center() # Obtém o centro da nuvem de pontos
pcd.translate(-pcd_center)
#o3d.visualization.draw_geometries([pcd]) # Visualiza a nuvem de pontos completa

In [ ]:
# Variáveis
retained_ratio = 0.2
nn = 16
std_multiplier = 10
voxel_size = 0.05

# 3.1. Sampling
#%% 3.1. Random Sampling Test
sampled_pcd = pcd.random_down_sample(retained_ratio)

#%% 3.2. Statistical outlier filter 
filtered_pcd, filtered_idx = pcd.remove_statistical_outlier(nn, std_multiplier)

#%% 3.3. Voxel downsampling
pcd_downsampled = filtered_pcd.voxel_down_sample(voxel_size = voxel_size)

#%% 3.4. Estimating normals
nn_distance = np.mean(pcd.compute_nearest_neighbor_distance())
print(nn_distance)
#setting the radius search to compute normals

radius_normals=nn_distance*4

pcd_downsampled.estimate_normals(search_param=o3d.geometry.KDTreeSearchParamHybrid(radius=radius_normals, max_nn=16), fast_normal_computation=True)



In [ ]:
front = [ 0.41218726271625999, -0.88460766947872382, 0.21810761461695488 ]
lookat = [ 1.0440935361781687, -0.24952285743042835, 0.26834577103162482 ]
up = [ -0.082609051253653878, 0.20211729830728198, 0.9758710685208043 ]
zoom = 0.69999999999999996

pcd = pcd_downsampled # Atualiza a nuvem de pontos

pt_to_plane_dist = 0.1 
ransac_n = 3
num_iterations = 1000


max_plane_idx = 10
nn_distance = np.mean(pcd.compute_nearest_neighbor_distance()) # Podemos colocar valores manuais
pt_to_plane_dist = nn_distance + np.std(pcd.compute_nearest_neighbor_distance()) # Podemos colocar valores manuais

In [ ]:
#%% 8. DBSCAN
segment_models = {}
segments = {}

rest = pcd

principal_min_cluster_points = 20

for i in range(max_plane_idx):
    colors = plt.get_cmap("tab20")(i)
    segment_models[i], inliers = rest.segment_plane(distance_threshold=pt_to_plane_dist,ransac_n=ransac_n,num_iterations=num_iterations)
    segments[i]=rest.select_by_index(inliers)
    
    labels = np.array(segments[i].cluster_dbscan(eps=pt_to_plane_dist*3, min_points=principal_min_cluster_points))
    rest = rest.select_by_index(inliers, invert=True)+segments[i].select_by_index(list(np.where(labels!=0)[0]))
    segments[i]=segments[i].select_by_index(list(np.where(labels==0)[0]))
    segments[i].paint_uniform_color(list(colors[:3]))
    print("pass",i,"/",max_plane_idx,"done.")


o3d.visualization.draw_geometries([segments[i] for i in range(max_plane_idx)],zoom=zoom, front=front, lookat=lookat,up=up)

In [ ]:
#%%2 9. DBSCAN Rest
rest_epsilon = 0.16
rest_min_cluster_points = 6

rest_db = rest

labels = np.array(rest_db.cluster_dbscan(eps=rest_epsilon, min_points=rest_min_cluster_points))
max_label = labels.max()
print(f"point cloud has {max_label + 1} clusters")

colors = plt.get_cmap("tab20")(labels / (max_label if max_label > 0 else 1))
colors[labels < 0] = 0
rest_db.colors = o3d.utility.Vector3dVector(colors[:, :3])


o3d.visualization.draw_geometries([rest],zoom=zoom, front=front, lookat=lookat,up=up) # Visualiza a nuvem de pontos restante com as cores dos clusters
o3d.visualization.draw_geometries([segments[i] for i in range(max_plane_idx)]+[rest],zoom=zoom, front=front, lookat=lookat,up=up) # Visualiza todos os segmentos e a nuvem de pontos restante

In [ ]:
# Salvar nuvem de pontos
xyz_segments=[]
for idx in segments:
    print(idx,segments[idx])
    a = np.asarray(segments[idx].points)
    N = len(a)
    b = idx*np.ones((N,3+1))
    b[:,:-1] = a
    xyz_segments.append(b)

rest_w_segments=np.hstack((np.asarray(rest.points),(labels+max_plane_idx).reshape(-1, 1)))
xyz_segments.append(rest_w_segments)
np.savetxt("RESULTS/" + input("Qual o nome que será dado ao arquivo?") + ".xyz", np.concatenate(xyz_segments), delimiter=";", fmt="%1.9f")

In [ ]:
# Calcular o Epsilon
from sklearn.neighbors import NearestNeighbors

def calculate_k_distance(X, k):
    neigh = NearestNeighbors(n_neighbors=k)
    nbrs = neigh.fit(X)
    distances, indices = nbrs.kneighbors(X)
    return sorted(distances[:, k-1], reverse=True)

def plot_k_distance(distances, k):
    plt.figure(figsize=(5, 5))
    plt.plot(list(range(1, len(distances) + 1)), distances)
    plt.xlabel('Points sorted by distance to kth nearest neighbor')
    plt.ylabel(f'{k}th nearest neighbor distance')
    plt.show()

# Suponha que 'pcd' seja sua nuvem de pontos e 'k' o número de vizinhos que você deseja considerar
k = 16
principal = np.asarray(pcd.points)
rest = np.asarray(rest.points)
distances_principal = calculate_k_distance(principal, k)
discances_rest = calculate_k_distance(rest, k)
plot_k_distance(distances_principal, k)
plot_k_distance(discances_rest, k)
